In [1]:
import pandas as pd
import numpy as np

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [87]:
# cf) data가 없는 zero_index 
zero_index = [13,  125,  345,  631,  655,  746,  850,  901, 1003, 1063, 1110, 
              1265, 1453, 1608, 1672, 1713, 1756, 2060, 2122, 2365, 2443, 2475,
              2587, 2603, 2740, 2787, 2853, 3226, 3308, 3464, 3570, 3637, 3687,
              3756, 3793, 3915, 4044, 4093, 4260, 4405, 4503, 4533, 4614, 4689,
              4899, 4961, 5140, 5169, 5235, 5338, 5373, 5473, 5558, 5630, 5722,
              5886, 6006, 6014, 6036, 6046, 6318, 6385, 6519, 6761, 6781, 6923,
              7288, 7388, 7515, 7527, 7593, 7755]
len(zero_index)

72

In [2]:
total_data = pd.read_csv('total_data.csv')
total_data

,SUBJECT_ID,ITEMID,CHARTTIME,TYPE
0,9,50882,2149-11-09,LAB
1,9,50931,2149-11-09,LAB
2,9,50971,2149-11-09,LAB
3,9,51275,2149-11-09,LAB
4,9,51288,2149-11-09,LAB
...,...,...,...,...
9221915,99985,51288,2181-03-06,LAB
9221916,99985,51279,2182-03-14,LAB
9221917,99985,51006,2182-03-14,LAB
9221918,99985,51221,2182-03-14,LAB


# 환자별 입원 기간에 대한 EDA
## 환자의 입원 기간별 비율 
- min : 0일, max : 4058일(11년)
- 0이상 ~ 100미만 stay : 4967명/7799명 (63.68)
- 100이상 ~ 200미만 stay : 509명/7799명 (6.52)
- 200이상 ~ 300미만 stay : 341명/7799명 (4.37)
- 300이상 ~ 400미만 stay : 417명/7799명 (5.34)
> 0일이상 400일 미만으로 머무른 환자 수는 6234명이며 전체 환자 수의 79.9%이다.

## 0이상 ~ 100미만에 대한 10단위 세분화
- 0이상 ~ 10미만 : 1723명/7799명 (22.0)
- 10이상 20미만 : 1300명/7799명 (16.6)
- 20이상 30미만 : 688명/7799명 (8.82)
- 30이상 40미만 : 425명/7799명 (5.44)

## 장단기 data 후보
- 후보1) interval : 1일, total : 10일 (10) -> 기존의 데이터 / 'x_(7727, 10, 4069).npy'
- 후보2) interval : 2일, total : 20일 (10) -> 'x_int2_(7727, 10, 4069).npy'
- 후보3) interval : 3일, total : 30일 (10) -> 'x_int3_(7727, 10, 4069).npy'
- 후보4) interval : 2일, total : 60일 (30) -> 'x_int2_(7727, 30, 4069).npy'
- 후보5) interval : 3일, total : 90일 (30) -> 'x_int3_(7727, 30, 4069).npy'

In [136]:
# stay를 위한 df
import datetime as dt

mn = total_data.pivot_table(columns=['SUBJECT_ID'],values=['CHARTTIME'],aggfunc='min').transpose()
mx = total_data.pivot_table(columns=['SUBJECT_ID'],values=['CHARTTIME'],aggfunc='max').transpose()
datecheck = pd.merge(mn, mx, on='SUBJECT_ID')
datecheck['CHARTTIME_x'] = pd.to_datetime(datecheck['CHARTTIME_x'])
datecheck['CHARTTIME_y'] = pd.to_datetime(datecheck['CHARTTIME_y'])
datecheck['stay']=(datecheck['CHARTTIME_y']-datecheck['CHARTTIME_x']).dt.days
datecheck

,CHARTTIME_x,CHARTTIME_y,stay
SUBJECT_ID,,,
9,2149-11-09,2149-11-15,6
21,2134-08-20,2135-02-08,172
31,2108-08-22,2108-08-31,9
36,2131-04-30,2134-05-20,1116
37,2183-08-21,2184-04-27,250
...,...,...,...
99863,2142-04-02,2142-05-08,36
99934,2110-02-28,2110-03-06,6
99937,2128-05-11,2128-05-20,9


In [127]:
# stay 0~4058일에 대한 10일 단위 구간화 
stay_df = pd.DataFrame(datecheck['stay'])
bins = list(range(0, 4200, 100))
bins_label = [str(x)+"이상 "+str(x+100)+"미만" for x in bins]
stay_df["level"] = pd.cut(stay_df['stay'], bins, right=False, labels=bins_label[:-1])
stay_df

,stay,level
SUBJECT_ID,,
9,6,0이상 100미만
21,172,100이상 200미만
31,9,0이상 100미만
36,1116,1100이상 1200미만
37,250,200이상 300미만
...,...,...
99863,36,0이상 100미만
99934,6,0이상 100미만
99937,9,0이상 100미만


In [118]:
# stay 0~100일에 대한 10일 단위 구간화 
stay100 = stay_df[stay_df['level']=='0이상 100미만'].drop(columns='level')
bins = list(range(0, 110, 10))
bins_label = [str(x)+"이상 "+str(x+10)+"미만" for x in bins]
stay100["level"] = pd.cut(stay100['stay'], bins, right=False, labels=bins_label[:-1])
stay100

,stay,level
SUBJECT_ID,,
9,6,0이상 10미만
31,9,0이상 10미만
41,32,30이상 40미만
101,11,10이상 20미만
103,20,20이상 30미만
...,...,...
99836,66,60이상 70미만
99863,36,30이상 40미만
99934,6,0이상 10미만


# 7727명만을 포함한 total_data_7727.csv

In [77]:
# total_data.csv -> 7727명을 포함한 total_data.csv
x = np.load('./x_(7799,10,4069).npy')

sub_sum = x.sum(axis=1).sum(axis=1)
sub_sum[sub_sum>1] = 1
sub_sum = pd.DataFrame(sub_sum)
sub_sum = sub_sum.reset_index()
sub_sum.columns = ['subid','value']
# sub_sum

zero_index = sub_sum[sub_sum['value']==0].index
sub_df = pd.DataFrame(total_data['SUBJECT_ID'].unique())
sub_df = sub_df.drop(index=zero_index)
subid_7727 = sub_df[0].values
# subid_7727

total_data = pd.read_csv('total_data.csv')
total_data2 = total_data[total_data['SUBJECT_ID'].isin(subid_7727)]
total_data2

# total_data2 -> csv로 저장
# total_data2.to_csv('total_data_7727.csv', index=False)

,SUBJECT_ID,ITEMID,CHARTTIME,TYPE
0,9,50882,2149-11-09,LAB
1,9,50931,2149-11-09,LAB
2,9,50971,2149-11-09,LAB
3,9,51275,2149-11-09,LAB
4,9,51288,2149-11-09,LAB
...,...,...,...,...
9221915,99985,51288,2181-03-06,LAB
9221916,99985,51279,2182-03-14,LAB
9221917,99985,51006,2182-03-14,LAB
9221918,99985,51221,2182-03-14,LAB


# int = 1, total = 10 (len = 10)
- 'x_(7727, 10, 4069).npy'로 저장

In [118]:
total_data2 = pd.read_csv('total_data_7727.csv')

dic_sub2idx = {}
for i, j in enumerate(total_data2['SUBJECT_ID'].unique()):
    dic_sub2idx[j] = i

dic_item2idx = {}
for i, j in enumerate(total_data['ITEMID'].sort_values().unique()): ## total_data2의 ITEMID 개수 : 4068개 
    dic_item2idx[j] = i

admission = pd.read_csv('/project/data/ADMISSIONS.csv')
admission = admission[admission['SUBJECT_ID'].isin(total_data2['SUBJECT_ID'].unique())]

# DISCHTIME열 시간 제거
import datetime as dt
admission['DISCHTIME'] = pd.to_datetime(admission['DISCHTIME'])
admission['DISCHTIME'] = admission['DISCHTIME'].dt.date

dic_sub2final_date = dict(admission.groupby('SUBJECT_ID')['DISCHTIME'].max())

len(dic_sub2idx)
len(dic_item2idx)
len(dic_sub2final_date)

7727

4069

7727

In [121]:
from datetime import timedelta
from tqdm import tqdm
import datetime

x = np.zeros((7727,10,4069))

IF = open('total_data_7727.csv','r')
line = IF.readline()
for line in tqdm(IF):
    ss = line.strip('\n').split(',')
    sub, item, charttime = int(ss[0]), int(ss[1]), datetime.date.fromisoformat(ss[2])
    subidx = dic_sub2idx[sub]
    itemidx = dic_item2idx[item]
    finaldate = dic_sub2final_date[sub]
    interval = (finaldate - charttime).days
    
    if (interval < 1) or (interval > 10): # interval : 1일이상 ~ 10일이하만 고려 
        pass
    else:
        dateidx = -((finaldate - charttime).days) + 10
        x[subidx, dateidx, itemidx] = 1

IF.close()
print(x.sum())
print(x.shape)

9169855it [00:14, 623455.64it/s]


2042722.0
(7727, 10, 4069)


In [122]:
# 기존에 만든 것과 sum이 같은지 확인 
t = np.load('x_(7727,10,4069).npy')
t.sum()

2042722.0

# int = 2, total = 20 (len = 10)
- 'x_int2_(7727, 10, 4069).npy'로 저장
- 7727명 중 data가 하나도 없는 환자 : 196명

- ★ 계산 note 
- dateidx = -(finaldate - charttime).days + 10
- (D-?) 0 ( 1 2 3 4 5 6 7 8 9 10 )
- (arr_index) 10 ( 9 8 7 6 5 4 3 2 1 0 )


- dateidx = -((finaldate - charttime).days)//2 + 10
- (finaldate - charttime) % 2 == 0 => 조건추가
- (D-?) = 0 ( 2 4 6 8 10 12 14 16 18 20 )
- (arr_index) = 10 ( 9 8 7 6 5 4 3 2 1 0 )

In [142]:
from datetime import timedelta
from tqdm import tqdm
import datetime

x = np.zeros((7727,10,4069))

IF = open('total_data_7727.csv','r')
line = IF.readline()
for line in tqdm(IF):
    ss = line.strip('\n').split(',')
    sub, item, charttime = int(ss[0]), int(ss[1]), datetime.date.fromisoformat(ss[2])
    subidx = dic_sub2idx[sub]
    itemidx = dic_item2idx[item]
    finaldate = dic_sub2final_date[sub]
    interval = (finaldate - charttime).days
    
    if (interval < 2) or (interval > 20): # interval : 2일이상 ~ 20일이하만 고려 
        pass
    else:
        if ((interval % 2) == 0):
            dateidx = -((finaldate - charttime).days)//2 + 10
            x[subidx, dateidx, itemidx] = 1

IF.close()
print(x.sum())
print(x.shape)

9169855it [00:14, 617069.65it/s]


1461410.0
(7727, 10, 4069)


In [143]:
# x save
# np.save('x_int2_(7727,10,4069).npy',x)

# x load
x_int2 = np.load("x_int2_(7727, 10, 4069).npy")

# int = 3, total = 30 (len = 10)
- 'x_int3_(7727, 10, 4069).npy'로 저장

In [130]:
from datetime import timedelta
from tqdm import tqdm
import datetime

x = np.zeros((7727,10,4069))

IF = open('total_data_7727.csv','r')
line = IF.readline()
for line in tqdm(IF):
    ss = line.strip('\n').split(',')
    sub, item, charttime = int(ss[0]), int(ss[1]), datetime.date.fromisoformat(ss[2])
    subidx = dic_sub2idx[sub]
    itemidx = dic_item2idx[item]
    finaldate = dic_sub2final_date[sub]
    interval = (finaldate - charttime).days
    
    if (interval < 3) or (interval > 30): # interval : 3일이상 ~ 30일이하만 고려 
        pass
    else:
        if ((interval % 3) == 0):
            dateidx = -((finaldate - charttime).days)//3 + 10
            x[subidx, dateidx, itemidx] = 1

IF.close()
print(x.sum())
print(x.shape)

9169855it [00:14, 640646.42it/s]


1095557.0
(7727, 10, 4069)


In [131]:
# x save
np.save('x_int3_(7727,10,4069).npy',x)

# x load
x_int2 = np.load("x_int3_(7727, 10, 4069).npy")

# int = 2, total = 60 (len = 30)
- 'x_int2_(7727,30,4069).npy'로 저장

- ★ 계산 note 
- dateidx = -(finaldate - charttime).days + 10
- (D-?) 0 ( 1 2 3 4 5 6 7 8 9 10 )
- (arr_index) 10 ( 9 8 7 6 5 4 3 2 1 0 )


- dateidx = -((finaldate - charttime).days)//2 + 30
- (finaldate - charttime) % 2 == 0 => 조건추가
- (D-?) = 0 ( 2 4 6 8 10 12 14 16 18 20 ... 58 60 )
- (arr_index) = 30 ( 29 28 27 26 25 24 ... 3 2 1 0 )

In [141]:
from datetime import timedelta
from tqdm import tqdm
import datetime

x = np.zeros((7727,30,4069))

IF = open('total_data_7727.csv','r')
line = IF.readline()
for line in tqdm(IF):
    ss = line.strip('\n').split(',')
    sub, item, charttime = int(ss[0]), int(ss[1]), datetime.date.fromisoformat(ss[2])
    subidx = dic_sub2idx[sub]
    itemidx = dic_item2idx[item]
    finaldate = dic_sub2final_date[sub]
    interval = (finaldate - charttime).days
    
    if (interval < 2) or (interval > 60): # interval : 2일이상 ~ 60일이하만 고려 
        pass
    else:
        if ((interval % 2) == 0):
            dateidx = -((finaldate - charttime).days)//2 + 30
            x[subidx, dateidx, itemidx] = 1

IF.close()
print(x.sum())
print(x.shape)

9169855it [00:15, 591364.86it/s]


1986925.0
(7727, 30, 4069)


In [136]:
# x save
# np.save('x_int2_(7727,30,4069).npy',x)

# x load
x_int2_30 = np.load("x_int2_(7727,30,4069).npy")

# int = 3, total = 90 (len = 30)
- 'x_int3_(7727, 30, 4069).npy'로 저장

- ★ 계산 note 
- dateidx = -(finaldate - charttime).days + 10
- (D-?) 0 ( 1 2 3 4 5 6 7 8 9 10 )
- (arr_index) 10 ( 9 8 7 6 5 4 3 2 1 0 )


- dateidx = -((finaldate - charttime).days)//3 + 30
- (finaldate - charttime) % 3 == 0 => 조건추가
- (D-?) = 0 ( 3 6 9 12 15 ... 87 90 )
- (arr_index) = 30 ( 29 28 27 26 25 24 ... 3 2 1 0 )

In [140]:
from datetime import timedelta
from tqdm import tqdm
import datetime

x = np.zeros((7727,30,4069))

IF = open('total_data_7727.csv','r')
line = IF.readline()
for line in tqdm(IF):
    ss = line.strip('\n').split(',')
    sub, item, charttime = int(ss[0]), int(ss[1]), datetime.date.fromisoformat(ss[2])
    subidx = dic_sub2idx[sub]
    itemidx = dic_item2idx[item]
    finaldate = dic_sub2final_date[sub]
    interval = (finaldate - charttime).days
    
    if (interval < 3) or (interval > 90): # interval : 3일이상 ~ 90일이하만 고려 
        pass
    else:
        if ((interval % 3) == 0):
            dateidx = -((finaldate - charttime).days)//3 + 30
            x[subidx, dateidx, itemidx] = 1

IF.close()
print(x.sum())
print(x.shape)

9169855it [00:14, 617086.22it/s]


1380745.0
(7727, 30, 4069)
